In [ ]:
import tensorflow as tf
print("GPU kullanılabilirliği: ", tf.test.is_gpu_available())
!pip install transformers
import numpy as np
import pandas as pd
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Input, Dense, Flatten, Concatenate, GlobalAveragePooling2D, Lambda
from tensorflow.keras.models import Model
from transformers import TFViTModel
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
data_path = '/content/drive/MyDrive/KaggleDataSets/celeb-df.zip'

In [ ]:
base_path = '/content/drive/MyDrive/KaggleDataSets/celeb-df_extracted/celeb-df'
train_path = os.path.join(base_path,'train ')
test_path  = os.path.join(base_path,'test')
val_path = os.path.join(base_path,'validation')

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(224,224),
    batch_size=32,
    class_mode='binary',
    shuffle = True
)
val_generator = val_datagen.flow_from_directory(
    val_path,
    target_size=(224,224),
    batch_size=32,
    class_mode='binary',
    shuffle = False
)
test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=(224,224),
    batch_size=32,
    class_mode='binary',
    shuffle = False
)

Found 16276 images belonging to 2 classes.
Found 4070 images belonging to 2 classes.


In [ ]:
print(f"Training Samples:{train_generator.samples}")
print(f"Validation Samples:{val_generator.samples}")
print(f"Test Generator Samples: {test_generator.samples}")
print(f"Sınıf Indeksleri", train_generator.class_indices)

Training Samples:5866
Validation Samples:16276
Test Generator Samples: 4070
Sınıf Indeksleri {'fake': 0, 'real': 1}


In [ ]:
vgg16_base = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
for layer in vgg16_base.layers:
    layer.trainable = False
vgg16_output = vgg16_base.output
vgg16_output = GlobalAveragePooling2D()(vgg16_output)

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
vit_model = TFViTModel.from_pretrained('google/vit-base-patch16-224')
vit_input = Input(shape=(224, 224, 3))
vit_transposed = Lambda(lambda x: tf.transpose(x, perm=[0, 3, 1, 2]))(vit_input)
vit_output = Lambda(lambda x: vit_model(x).last_hidden_state[:, 0, :], output_shape=(768,))(vit_transposed)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/69.7k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFViTModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing TFViTModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFViTModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFViTModel were not initialized from the PyTorch model and are newly initialized: ['vit.pooler.dense.weight', 'vit.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
combined = Concatenate()([vgg16_output, vit_output])
x = Dense(128, activation='relu')(combined)
x = Dense(1, activation='sigmoid')(x)

In [ ]:
model = Model(inputs=[vgg16_base.input, vit_input], outputs=x)

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 224, 224, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1_conv1 (Conv2D)     │ (None, 224, 224, 64)   │          1,792 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1_conv2 (Conv2D)     │ (None, 224, 224, 64)   │         36,928 │ block1_conv1[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1_pool               │ (None, 112, 112, 64)   │              0 │ block1_conv2[0][0]     │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block2_conv1 (Conv2D)     │ (None, 112, 112, 128)  │         73,856 │ block1_pool[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block2_conv2 (Conv2D)     │ (None, 112, 112, 128)  │        147,584 │ block2_conv1[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block2_pool               │ (None, 56, 56, 128)    │              0 │ block2_conv2[0][0]     │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block3_conv1 (Conv2D)     │ (None, 56, 56, 256)    │        295,168 │ block2_pool[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block3_conv2 (Conv2D)     │ (None, 56, 56, 256)    │        590,080 │ block3_conv1[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block3_conv3 (Conv2D)     │ (None, 56, 56, 256)    │        590,080 │ block3_conv2[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block3_pool               │ (None, 28, 28, 256)    │              0 │ block3_conv3[0][0]     │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block4_conv1 (Conv2D)     │ (None, 28, 28, 512)    │      1,180,160 │ block3_pool[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block4_conv2 (Conv2D)     │ (None, 28, 28, 512)    │      2,359,808 │ block4_conv1[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block4_conv3 (Conv2D)     │ (None, 28, 28, 512)    │      2,359,808 │ block4_conv2[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block4_pool               │ (None, 14, 14, 512)    │              0 │ block4_conv3[0][0]     │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block5_conv1 (Conv2D)     │ (None, 14, 14, 512)    │      2,359,808 │ block4_pool[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block5_conv2 (Conv2D)     │ (None, 14, 14, 512)    │      2,359,808 │ block5_conv1[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block5_conv3 (Conv2D)

 Total params: 14,878,785 (56.76 MB)

 Trainable params: 164,097 (641.00 KB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [ ]:
def custom_generator(generator):
    while True:
        images, labels = next(generator)
        yield (images, images), labels

In [ ]:
train_dataset = tf.data.Dataset.from_generator(
    lambda: custom_generator(train_generator),
    output_signature=(
        (tf.TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32),
         tf.TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32)),
        tf.TensorSpec(shape=(None,), dtype=tf.float32)
    )
)

val_dataset = tf.data.Dataset.from_generator(
    lambda: custom_generator(val_generator),
    output_signature=(
        (tf.TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32),
         tf.TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32)),
        tf.TensorSpec(shape=(None,), dtype=tf.float32)
    )
)

In [ ]:
print(model.input)

[<KerasTensor shape=(None, 224, 224, 3), dtype=float32, sparse=False, name=keras_tensor>, <KerasTensor shape=(None, 224, 224, 3), dtype=float32, sparse=False, name=keras_tensor_20>]


In [ ]:
history = model.fit(
    train_dataset,
    epochs=10,
    validation_data=val_dataset,
    steps_per_epoch=train_generator.samples // 32,
    validation_steps=val_generator.samples // 32
)

Epoch 1/10
183/183 ━━━━━━━━━━━━━━━━━━━━ 9815s 54s/step - accuracy: 1.0000 - loss: 9.4485e-06 - val_accuracy: 0.0950 - val_loss: 19.3087
Epoch 2/10
183/183 ━━━━━━━━━━━━━━━━━━━━ 428s 2s/step - accuracy: 1.0000 - loss: 1.1165e-08 - val_accuracy: 0.0963 - val_loss: 19.2875
Epoch 3/10
183/183 ━━━━━━━━━━━━━━━━━━━━ 431s 2s/step - accuracy: 1.0000 - loss: 1.1851e-08 - val_accuracy: 0.0963 - val_loss: 19.2864
Epoch 4/10
183/183 ━━━━━━━━━━━━━━━━━━━━ 411s 2s/step - accuracy: 1.0000 - loss: 9.1721e-09 - val_accuracy: 0.0963 - val_loss: 19.2878
Epoch 5/10
183/183 ━━━━━━━━━━━━━━━━━━━━ 406s 2s/step - accuracy: 1.0000 - loss: 1.1037e-08 - val_accuracy: 0.0963 - val_loss: 19.2898
Epoch 6/10
183/183 ━━━━━━━━━━━━━━━━━━━━ 403s 2s/step - accuracy: 1.0000 - loss: 1.0151e-08 - val_accuracy: 0.0963 - val_loss: 19.2921
Epoch 7/10
183/183 ━━━━━━━━━━━━━━━━━━━━ 442s 2s/step - accuracy: 1.0000 - loss: 1.7075e-08 - val_accuracy: 0.0963 - val_loss: 19.2952
Epoch 8/10
183/183 ━━━━━━━━━━━━━━━━━━━━ 403s 2s/step - accur

In [ ]:
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

NameError: name 'plt' is not defined

In [ ]:
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
test_loss, test_accuracy = model.evaluate([test_generator, test_generator])
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")